# Load data

In [2]:
import pandas as pd

df = pd.read_csv('data.csv')
df = df[:100]
df.head()

,id,sentence,date,birthday
0,100022_0,Болеет СД 2 типа в течении 5 лет.,2010-12-14 18:33:23,1957-01-01
1,100085_0,"Считает себя больной с 1993 года, когда перене...",2010-12-14 21:06:48,1950-11-05
2,100085_4,В мае 2009 года в СибГМУ проводилась РЧ-аблаци...,2010-12-14 21:06:48,1950-11-05
3,100085_5,"Ухудшение в конце 2009, начале 2010 года, когд...",2010-12-14 21:06:48,1950-11-05
4,100085_7,В апреле 2010 года проходила лечение в отд НРС...,2010-12-14 21:06:48,1950-11-05


# Parse single sentence

In [ ]:
from TimeExpressions import TimeProcessor

processor = TimeProcessor.TimeProcessor()

In [14]:
doc = processor.process("Болеет СД 2 типа в течении 5 лет .", '2010-05-17 00:00:00', '1944-04-19')
print([('ВК: '+ent.text, 'Метка: '+str(ent._.timestamp), 'Нормальная форма: '+str(ent._.normal_form), 'Событие: '+ent._.event) for ent in doc[0].ents])

[('ВК: в течении 5 лет', 'Метка: 2', 'Нормальная форма: [datetime.datetime(2005, 5, 17, 0, 0), datetime.datetime(2010, 5, 17, 0, 0)]', 'Событие: Болеет СД 2 типа')]


# Parse set of sentences and convert result to pandas

In [5]:
docs = processor.process(df.sentence.values, df.date.values, df.birthday.values)

time_expr = []
stamps = []
events = []
normal_forms = []

for doc in docs:
    time_expr.append([ent.text for ent in doc.ents])
    stamps.append([ent._.timestamp for ent in doc.ents])
    events.append([ent._.event for ent in doc.ents])
    normal_forms.append([ent._.normal_form for ent in doc.ents])

In [6]:
time_expr_pred = [', '.join(expr) for expr in time_expr]
df['time_expr'] = time_expr_pred

In [7]:
# convert datetime objects to string
flat_list = normal_forms
for i in range(len(flat_list)):
    if type(flat_list[i]) == list:
        for j in range(len(flat_list[i])):
            if type(flat_list[i][j]) == list:
                flat_list[i][j] = ' – '.join([str(form.date()) for form in flat_list[i][j] if form is not None])
        flat_list[i] = ', '.join([str(form) for form in flat_list[i]])

df['normal_form'] = [str(i) for i in flat_list]

In [8]:
for stamp in stamps:
    for i in range(len(stamp)):
        stamp[i] = str(stamp[i])

stamps_pred = [', '.join(stamp) for stamp in stamps]
df['stamp'] = stamps_pred

In [9]:
events_pred = [', '.join(event) for event in events]
events_pred = [event.replace(' - ', '-') for event in events_pred]
df['event'] = events_pred

In [10]:
df.head()

,id,sentence,date,birthday,time_expr,normal_form,stamp,event
0,100022_0,Болеет СД 2 типа в течении 5 лет.,2010-12-14 18:33:23,1957-01-01,в течении 5 лет,2005-12-17 – 2010-12-17,2,Болеет СД 2 типа
1,100085_0,"Считает себя больной с 1993 года, когда перене...",2010-12-14 21:06:48,1950-11-05,с 1993 года,1993-01-01,2,перенесла гинекологическую операцию
2,100085_4,В мае 2009 года в СибГМУ проводилась РЧ - абла...,2010-12-14 21:06:48,1950-11-05,В мае 2009 года,2009-05-01,1,проводилась РЧ-аблация по поводу ЖЭ
3,100085_5,"Ухудшение в конце 2009, начале 2010 года, когд...",2010-12-14 21:06:48,1950-11-05,"в конце 2009 , начале 2010",2009-01-01 – 2010-01-01,1,Ухудшение
4,100085_7,В апреле 2010 года проходила лечение в отд НРС...,2010-12-14 21:06:48,1950-11-05,"В апреле 2010 года, за сутки","2010-04-01, None","1, 2","проходила лечение в отд НРС, 1 экстрасистола"
